# Project Summary

## Business Context

In the highly competitive e-commerce space, delivering excellent customer service is crucial for sustaining growth and customer loyalty. **Flipkart**, as one of the largest e-commerce platforms, focuses on enhancing customer satisfaction to differentiate itself from competitors.

The dataset used in this project captures customer interactions, feedback, and satisfaction scores across various support channels at Flipkart. By analyzing these interactions, the goal is to:

- Identify key drivers of customer satisfaction  
- Understand performance across different customer service teams  
- Develop strategies to improve the overall service experience  

Understanding the factors that influence customer satisfaction enables Flipkart to resolve customer issues faster and tailor support strategies to meet diverse customer expectations. This directly helps in optimizing agent performance and improving satisfaction metrics such as the **CSAT score**, ultimately leading to increased brand loyalty and customer retention.

---

## Objective

- Identify key drivers of customer satisfaction (important features)
- Develop a predictive model to estimate the **Customer Satisfaction Score (CSAT)**
- Propose strategies to improve the overall customer service experience

---

## Dataset Overview

The dataset **`Customer_support_data.csv`** contains multiple records from a customer support system. Each row represents a unique customer interaction, along with metadata such as agent details, response time, product information, and the final satisfaction score.

- **Target Variable**: CSAT Score  
- **CSAT Range**:  
  - 1 → Very Dissatisfied  
  - 5 → Very Satisfied  

The primary objective is to leverage the available features to accurately predict the CSAT score and gain insights into service quality.

---

## Feature-wise Explanation

| Feature Name | Description |
|-------------|-------------|
| Unique id | Unique identifier for each record |
| Channel name | Name of the customer service channel |
| Category | Category of the interaction |
| Sub-category | Sub-category of the interaction |
| Customer Remarks | Feedback provided by the customer |
| Order id | Identifier for the associated order |
| Order date time | Date and time of the order |
| Issue reported at | Timestamp when the issue was reported |
| Issue responded | Timestamp when the issue was responded to |
| Survey response date | Date of customer survey response |
| Customer city | City of the customer |
| Product category | Category of the product |
| Item price | Price of the purchased item |
| Connected handling time | Time taken to handle the interaction |
| Agent name | Name of the customer service agent |
| Supervisor | Supervisor overseeing the agent |
| Manager | Manager of the service team |
| Tenure Bucket | Bucket categorizing agent tenure |
| Agent Shift | Shift timing of the agent |
| CSAT Score | Customer Satisfaction score (target variable) |

---

## Real-World Use Cases

- Develop a system to **predict customers who need immediate query resolution**
- Identify customers who are **most likely to give low CSAT scores**
- Enable **live monitoring of service quality**
- **Flag at-risk interactions** in real time
- Identify **issue types, agents, or teams** contributing to poor satisfaction
- Gain a deeper understanding of data patterns and feature importance

---

## Know Your Data and Features

A thorough understanding of the dataset and its features is essential for:
- Building reliable predictive models
- Interpreting model outputs
- Translating insights into actionable business strategies


In [1]:
import pandas as pd
import numpy as np

***Loading the dataset***

In [2]:
df = pd.read_csv("Customer_support_data.csv")
df.columns

Index(['Unique id', 'channel_name', 'category', 'Sub-category',
       'Customer Remarks', 'Order_id', 'order_date_time', 'Issue_reported at',
       'issue_responded', 'Survey_response_Date', 'Customer_City',
       'Product_category', 'Item_price', 'connected_handling_time',
       'Agent_name', 'Supervisor', 'Manager', 'Tenure Bucket', 'Agent Shift',
       'CSAT Score'],
      dtype='object')

We also positive customer reviews, assuming all instances were collected in the same way, then the reviews must be collected after the issue was resolved. So we can't use customer reviews for predictive modeling. 

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                85907 non-null  object 
 1   channel_name             85907 non-null  object 
 2   category                 85907 non-null  object 
 3   Sub-category             85907 non-null  object 
 4   Customer Remarks         28742 non-null  object 
 5   Order_id                 67675 non-null  object 
 6   order_date_time          17214 non-null  object 
 7   Issue_reported at        85907 non-null  object 
 8   issue_responded          85907 non-null  object 
 9   Survey_response_Date     85907 non-null  object 
 10  Customer_City            17079 non-null  object 
 11  Product_category         17196 non-null  object 
 12  Item_price               17206 non-null  float64
 13  connected_handling_time  242 non-null    float64
 14  Agent_name            

***We have got 13.1+ MB of memory usage, we will try to optimise this. Along the way we will be learning pandas chaining techniques.***

We have many features with object datatype, so there will be opporunities of reducing memory usage. Also there are many missing values, we need to handle this carefully.

In [ ]:
df.memory_usage(deep = True).sum()   

***So now instead of 13.1 MB we have 96 MB of memory usage, we will try to optimise this.***

In [5]:
df.describe()

,Item_price,connected_handling_time,CSAT Score
count,17206.000000,242.000000,85907.000000
mean,5660.774846,462.400826,4.242157
std,12825.728411,246.295037,1.378903
min,0.000000,0.000000,1.000000
25%,392.000000,293.000000,4.000000
50%,979.000000,427.000000,5.000000
75%,2699.750000,592.250000,5.000000
max,164999.000000,1986.000000,5.000000


In [6]:
df.duplicated()

0        False
1        False
2        False
3        False
4        False
         ...  
85902    False
85903    False
85904    False
85905    False
85906    False
Length: 85907, dtype: bool

***Missing/Null Values***

In [7]:
(df.isnull().mean()*100).sort_values(ascending=False).round(2)

connected_handling_time    99.72
Customer_City              80.12
Product_category           79.98
Item_price                 79.97
order_date_time            79.96
Customer Remarks           66.54
Order_id                   21.22
Unique id                   0.00
Sub-category                0.00
category                    0.00
channel_name                0.00
Issue_reported at           0.00
issue_responded             0.00
Survey_response_Date        0.00
Agent_name                  0.00
Supervisor                  0.00
Manager                     0.00
Tenure Bucket               0.00
Agent Shift                 0.00
CSAT Score                  0.00
dtype: float64

***Know your Variables/Attributes***

| Feature Name             | Description                                                    | Relevance to CSAT Prediction                                                                 |
|--------------------------|----------------------------------------------------------------|----------------------------------------------------------------------------------------------|
| Unique id                | Unique identifier for each record                              | Not used in modeling, but helps track individual tickets.                                   |
| Channel name             | Name of the customer service channel                           | Different channels may affect response speed and customer experience.                       |
| Category                 | Category of the interaction                                    | Issue type may influence satisfaction (e.g., technical issues might be harder to resolve).  |
| Sub-category             | Sub-category of the interaction                                | Granular issue details could reveal patterns in satisfaction.                               |
| Customer Remarks         | Feedback provided by the customer                              | Text data for NLP; sentiment may strongly correlate with CSAT. But it can't be used for predictive modeling as it was collected after we got the CSAT score from customers.  |
| Order id                 | Identifier for the order associated with the interaction       | May link to order details, but less directly relevant unless joined with other data.        |
| Order date time          | Date and time of the order                                     | Can derive time gaps (e.g., order-to-issue delay) that might affect satisfaction.           |
| Issue reported at        | Timestamp when the issue was reported                          | Used to calculate response time, which likely impacts CSAT.                                 |
| Issue responded          | Timestamp when the issue was responded to                      | Key for response time calculation; faster responses may lead to higher CSAT.                |
| Survey response date     | Date of the customer survey response                           | Helps understand feedback timing, but may not directly predict CSAT.                        |
| Customer city            | City of the customer                                           | Location-based trends may exist; high cardinality may need careful encoding.                |
| Product category         | Category of the product                                        | Product type may influence expectations and satisfaction.                                   |
| Item price               | Price of the item                                              | Higher prices might raise expectations, affecting CSAT.                                     |
| Connected handling time  | Time taken to handle the interaction                           | Longer times may indicate complex issues or inefficiency, lowering CSAT.                    |
| Agent name               | Name of the customer service agent                             | Agent performance may impact CSAT; high cardinality noted from df.head().                   |
| Supervisor               | Name of the supervisor                                         | Useful for team-level performance trends.                                                    |
| Manager                  | Name of the manager                                            | May reveal management-level impacts on service quality.                                     |
| Tenure Bucket            | Bucket categorizing agent tenure                               | More experienced agents might resolve issues better, improving CSAT.                        |
| Agent Shift              | Shift timing of the agent                                      | Night shifts might have lower CSAT due to staffing or fatigue.                              |
| CSAT Score               | Customer Satisfaction (CSAT) score                             | The variable to predict; df.describe() suggests possible imbalance (e.g., skewed to 4–5).   |


In [8]:
df[~df["Item_price"].isnull()]["Item_price"]

11         434.0
16        1299.0
19       15990.0
24        1099.0
25       99999.0
          ...   
85878       25.0
85881      599.0
85886      998.0
85900      193.0
85901    23999.0
Name: Item_price, Length: 17206, dtype: float64

## Data Wrangling

In [9]:
(df
 .select_dtypes(int)
 )

,CSAT Score
0,5
1,5
2,5
3,5
4,5
...,...
85902,4
85903,5
85904,5
85905,4


In [10]:
df['CSAT Score'].value_counts()

CSAT Score
5    59617
1    11230
4    11219
3     2558
2     1283
Name: count, dtype: int64

In [11]:
df['CSAT Score'].value_counts(normalize=True)

CSAT Score
5    0.693971
1    0.130723
4    0.130595
3    0.029776
2    0.014935
Name: proportion, dtype: float64

In [12]:

#chaining
(df
 .select_dtypes(int)
 .describe()
 )

,CSAT Score
count,85907.000000
mean,4.242157
std,1.378903
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [13]:
(df
 .astype({'CSAT Score':'int8'})
 .select_dtypes([int,'int8'])
 .describe()
 )

,CSAT Score
count,85907.000000
mean,4.242157
std,1.378903
min,1.000000
25%,4.000000
50%,5.000000
75%,5.000000
max,5.000000


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                85907 non-null  object 
 1   channel_name             85907 non-null  object 
 2   category                 85907 non-null  object 
 3   Sub-category             85907 non-null  object 
 4   Customer Remarks         28742 non-null  object 
 5   Order_id                 67675 non-null  object 
 6   order_date_time          17214 non-null  object 
 7   Issue_reported at        85907 non-null  object 
 8   issue_responded          85907 non-null  object 
 9   Survey_response_Date     85907 non-null  object 
 10  Customer_City            17079 non-null  object 
 11  Product_category         17196 non-null  object 
 12  Item_price               17206 non-null  float64
 13  connected_handling_time  242 non-null    float64
 14  Agent_name            

In [15]:
(df
 .astype({'CSAT Score':'int64'})
 .memory_usage(deep=True)
 .sum()    # last was 96483983
 )

np.int64(87053307)

In [16]:
(df
 .astype({'CSAT Score':'int8'})
 .memory_usage(deep=True)
 .sum()    # last was 87053307
 )

np.int64(86451958)

In [17]:
(df
 #.astype({'CSAT Score':'int8'})
 #.memory_usage(deep=True)
 #.sum()    # intitial wa 96483983
 .select_dtypes([float])
 .describe()
 )

,Item_price,connected_handling_time
count,17206.000000,242.000000
mean,5660.774846,462.400826
std,12825.728411,246.295037
min,0.000000,0.000000
25%,392.000000,293.000000
50%,979.000000,427.000000
75%,2699.750000,592.250000
max,164999.000000,1986.000000


In [18]:

(df
 .connected_handling_time
 .value_counts(dropna=False, normalize=True)
 )

connected_handling_time
NaN      0.997183
299.0    0.000035
301.0    0.000035
418.0    0.000035
282.0    0.000035
           ...   
63.0     0.000012
281.0    0.000012
856.0    0.000012
353.0    0.000012
505.0    0.000012
Name: proportion, Length: 212, dtype: float64

Connected_handling_type have a lot of missing values, we will be dropping them

In [19]:
(df
 .Item_price
 .value_counts(dropna=False, normalize=True)
 )

Item_price
NaN        0.799714
999.0      0.006868
1499.0     0.003795
899.0      0.003597
1299.0     0.003504
             ...   
1701.0     0.000012
1220.0     0.000012
1683.0     0.000012
28500.0    0.000012
1093.0     0.000012
Name: proportion, Length: 2790, dtype: float64

In [20]:
# where are they missing
(df
 .query('connected_handling_time.isna()')    # using query gives us that part of df with missing connected_handling_time
 )

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,NaN,NaN,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,NaN,NaN,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,NaN,NaN,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,NaN,NaN,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,NaN,NaN,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85902,505ea5e7-c475-4fac-ac36-1d19a4cb610f,Inbound,Refund Related,Refund Enquiry,NaN,1b5a2b9c-a95f-405f-a42e-5b1b693f3dc9,NaN,30/08/2023 23:20,31/08/2023 07:22,31-Aug-23,NaN,NaN,NaN,NaN,Brandon Leon,Ethan Tan,William Kim,On Job Training,Morning,4
85903,44b38d3f-1523-4182-aba2-72917586647c,Inbound,Order Related,Seller Cancelled Order,Supported team customer executive good,d0e8a817-96d5-4ace-bb82-adec50398e22,NaN,31/08/2023 08:15,31/08/2023 08:17,31-Aug-23,NaN,NaN,NaN,NaN,Linda Foster,Noah Patel,Emily Chen,>90,Morning,5
85904,723bce2c-496c-4aa8-a64b-ca17004528f0,Inbound,Order Related,Order status enquiry,need to improve with proper details.,bdefe788-ccec-4eda-8ca4-51045e68db8a,NaN,31/08/2023 18:57,31/08/2023 19:02,31-Aug-23,NaN,NaN,NaN,NaN,Kimberly Martinez,Aiden Patel,Olivia Tan,On Job Training,Evening,5
85905,707528ee-6873-4192-bfa9-a491f1c08ab5,Inbound,Feedback,UnProfessional Behaviour,NaN,a031ec28-0c5e-450e-95b2-592342c40bc4,NaN,31/08/2023 19:59,31/08/2023 20:00,31-Aug-23,NaN,NaN,NaN,NaN,Daniel Martin,Olivia Suzuki,Olivia Tan,>90,Morning,4


In [21]:

(df
 .query('Item_price.isna()')
 .shape
 )

(68701, 20)

In [22]:

(df
 .astype({'CSAT Score':'int8'})
 .drop(columns=['Item_price','connected_handling_time'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(85077446)

**Objects**

In [23]:

(df
 .select_dtypes(object)
 )

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85902,505ea5e7-c475-4fac-ac36-1d19a4cb610f,Inbound,Refund Related,Refund Enquiry,NaN,1b5a2b9c-a95f-405f-a42e-5b1b693f3dc9,NaN,30/08/2023 23:20,31/08/2023 07:22,31-Aug-23,NaN,NaN,Brandon Leon,Ethan Tan,William Kim,On Job Training,Morning
85903,44b38d3f-1523-4182-aba2-72917586647c,Inbound,Order Related,Seller Cancelled Order,Supported team customer executive good,d0e8a817-96d5-4ace-bb82-adec50398e22,NaN,31/08/2023 08:15,31/08/2023 08:17,31-Aug-23,NaN,NaN,Linda Foster,Noah Patel,Emily Chen,>90,Morning
85904,723bce2c-496c-4aa8-a64b-ca17004528f0,Inbound,Order Related,Order status enquiry,need to improve with proper details.,bdefe788-ccec-4eda-8ca4-51045e68db8a,NaN,31/08/2023 18:57,31/08/2023 19:02,31-Aug-23,NaN,NaN,Kimberly Martinez,Aiden Patel,Olivia Tan,On Job Training,Evening
85905,707528ee-6873-4192-bfa9-a491f1c08ab5,Inbound,Feedback,UnProfessional Behaviour,NaN,a031ec28-0c5e-450e-95b2-592342c40bc4,NaN,31/08/2023 19:59,31/08/2023 20:00,31-Aug-23,NaN,NaN,Daniel Martin,Olivia Suzuki,Olivia Tan,>90,Morning


In [24]:
# lets inspect channel name
(df
 .channel_name
 .value_counts(dropna=False, normalize=True))    #don't drop the nulls

channel_name
Inbound    0.793207
Outcall    0.171604
Email      0.035189
Name: proportion, dtype: float64

In [25]:
# ets make channel name as category
(df
 .astype({'CSAT Score':'int8', 'channel_name':'category'})
 .drop(columns=['Item_price','connected_handling_time'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )


np.int64(80358881)

In [26]:

# lets check category col
(df
 .category
 .value_counts(dropna=False, normalize=True))

category
Returns               0.513311
Order Related         0.270234
Refund Related        0.052964
Product Queries       0.042977
Shopzilla Related     0.032500
Payments related      0.027087
Feedback              0.026703
Cancellation          0.025749
Offers & Cashback     0.005587
Others                0.001152
App/website           0.000978
Onboarding related    0.000757
Name: proportion, dtype: float64

In [27]:
(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category'})
 .drop(columns=['Item_price','connected_handling_time'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(75366393)

In [28]:
(df
 .select_dtypes([object])
 .describe()
)

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift
count,85907,85907,85907,85907,28742,67675,17214,85907,85907,85907,17079,17196,85907,85907,85907,85907,85907
unique,85907,3,12,57,18231,67675,13766,30923,30262,31,1782,9,1371,40,6,5,5
top,07c7a878-0d5a-42e0-97ef-de59abec0238,Inbound,Returns,Reverse Pickup Enquiry,Good,3230db30-f8da-4c44-8636-ec76d1d3d4f3,09/08/2023 11:55,13/08/2023 10:40,28/08/2023 00:00,28-Aug-23,HYDERABAD,Electronics,Wendy Taylor,Carter Park,John Smith,>90,Morning
freq,1,68142,44097,22389,1390,1,7,13,3378,3452,722,4706,429,4273,25261,30660,41426


In [29]:
(df['Sub-category']
 .value_counts(dropna=False, normalize=True)
 )

Sub-category
Reverse Pickup Enquiry              0.260619
Return request                      0.099212
Delayed                             0.086000
Order status enquiry                0.080576
Installation/demo                   0.047912
Fraudulent User                     0.047819
Product Specific Information        0.041778
Refund Enquiry                      0.031022
Wrong                               0.030230
Missing                             0.029753
UnProfessional Behaviour            0.026703
General Enquiry                     0.022641
Not Needed                          0.022350
Service Centres Related             0.021826
Refund Related Issues               0.020953
Invoice request                     0.017053
Online Payment Issues               0.012560
Seller Cancelled Order              0.012327
Priority delivery                   0.011315
Exchange / Replacement              0.010430
Customer Requested Modifications    0.009371
Payment related Queries             0.0086

In [30]:
(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'})
 .drop(columns=['Item_price','connected_handling_time'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(69748221)

Supervisor	Manager	Tenure Bucket	Agent Shift

In [31]:

(df['Supervisor']
 .value_counts(dropna=False)
 )

Supervisor
Carter Park         4273
Elijah Yamaguchi    3855
Zoe Yamamoto        3551
Nathan Patel        3549
Noah Patel          3434
Ava Wong            3373
Emma Park           3290
Mia Patel           3279
Evelyn Kimura       2974
Aiden Patel         2892
Madison Kim         2772
William Park        2692
Scarlett Chen       2676
Emily Yamashita     2581
Jackson Park        2571
Brayden Wong        2540
Logan Lee           2466
Olivia Wang         2233
Abigail Suzuki      2222
Olivia Suzuki       2193
Mason Gupta         2109
Lily Chen           1869
Ethan Tan           1831
Wyatt Kim           1727
Austin Johnson      1723
Sophia Sato         1671
Landon Tanaka       1635
Ethan Nakamura      1584
Jacob Sato          1501
Amelia Tanaka       1364
Alexander Tanaka    1320
Lucas Singh         1285
Dylan Kim           1197
Charlotte Suzuki    1172
Harper Wong         1119
Isabella Wong       1081
Layla Taniguchi      924
Mia Yamamoto         625
Oliver Nguyen        417
Sophia Chen   

In [32]:
(df['Manager']
 .value_counts(dropna=False)
 )

Manager
John Smith         25261
Michael Lee        17600
Jennifer Nguyen    15897
Emily Chen         14062
William Kim         8568
Olivia Tan          4519
Name: count, dtype: int64

In [33]:
(df['Agent Shift']
 .value_counts(dropna=False)
 )

Agent Shift
Morning      41426
Evening      33677
Afternoon     5840
Split         3648
Night         1316
Name: count, dtype: int64

In [34]:
(df['Tenure Bucket']
 .value_counts(dropna=False, normalize=True)
 )

Tenure Bucket
>90                0.356898
On Job Training    0.297100
31-60              0.135786
0-30               0.131747
61-90              0.078469
Name: proportion, dtype: float64

In [35]:
(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'
          , 'Supervisor':'category','Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category'})
 .drop(columns=['Item_price','connected_handling_time'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(50077908)

Now we can see some significant drop

In [36]:
(df
 .Survey_response_Date
 .value_counts(dropna= False)
 )

Survey_response_Date
28-Aug-23    3452
27-Aug-23    3379
17-Aug-23    3193
29-Aug-23    3188
07-Aug-23    3014
22-Aug-23    3005
08-Aug-23    3000
26-Aug-23    2990
10-Aug-23    2989
24-Aug-23    2962
23-Aug-23    2944
14-Aug-23    2927
11-Aug-23    2894
16-Aug-23    2885
12-Aug-23    2841
09-Aug-23    2823
15-Aug-23    2819
13-Aug-23    2758
30-Aug-23    2757
18-Aug-23    2605
25-Aug-23    2550
04-Aug-23    2523
21-Aug-23    2510
19-Aug-23    2496
06-Aug-23    2490
03-Aug-23    2467
31-Aug-23    2389
02-Aug-23    2330
05-Aug-23    2283
01-Aug-23    2231
20-Aug-23    2213
Name: count, dtype: int64

We are going to drop this, because it is not a useful feature for our use case.

In [37]:

(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'
          , 'Supervisor':'category','Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(45095302)

In [38]:

(df 
 .select_dtypes(object)
 .describe()
 )

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift
count,85907,85907,85907,85907,28742,67675,17214,85907,85907,85907,17079,17196,85907,85907,85907,85907,85907
unique,85907,3,12,57,18231,67675,13766,30923,30262,31,1782,9,1371,40,6,5,5
top,07c7a878-0d5a-42e0-97ef-de59abec0238,Inbound,Returns,Reverse Pickup Enquiry,Good,3230db30-f8da-4c44-8636-ec76d1d3d4f3,09/08/2023 11:55,13/08/2023 10:40,28/08/2023 00:00,28-Aug-23,HYDERABAD,Electronics,Wendy Taylor,Carter Park,John Smith,>90,Morning
freq,1,68142,44097,22389,1390,1,7,13,3378,3452,722,4706,429,4273,25261,30660,41426


In [39]:
# lets look at product category
(df 
 .Product_category
 .value_counts(dropna=False, normalize=True)
 )

Product_category
NaN                            0.799830
Electronics                    0.054780
LifeStyle                      0.047936
Books & General merchandise    0.038681
Mobile                         0.020464
Home                           0.015459
Home Appliences                0.015133
Furniture                      0.005483
Affiliates                     0.001932
GiftCard                       0.000303
Name: proportion, dtype: float64

In [40]:

# lets look at the customer city
(df 
 .Customer_City
 .value_counts(dropna= False, normalize=True)
 )

Customer_City
NaN            0.801192
HYDERABAD      0.008404
NEW DELHI      0.008009
PUNE           0.005064
MUMBAI         0.004726
                 ...   
JAGGAYYAPET    0.000012
CONTAI         0.000012
ALANGUDI       0.000012
CURCHOREM      0.000012
PATTIKONDA     0.000012
Name: proportion, Length: 1783, dtype: float64

In [41]:
# lets look at the customer city
(df 
 .order_date_time
 .value_counts(dropna= False, normalize=True)
 )

order_date_time
NaN                 0.799621
09/08/2023 11:55    0.000081
05/08/2023 20:14    0.000070
05/08/2023 13:07    0.000058
03/08/2023 12:06    0.000058
                      ...   
29/08/2023 12:28    0.000012
29/08/2023 13:12    0.000012
22/08/2023 20:59    0.000012
27/07/2023 21:27    0.000012
20/08/2023 13:45    0.000012
Name: proportion, Length: 13767, dtype: float64

In [42]:

(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'
          , 'Supervisor':'category','Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category','Customer_City'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(38663414)

In [43]:
# Since we are using this dataset for model training, we can simply drop unique id , order_id and order_date_time
(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'
          , 'Supervisor':'category','Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )


np.int64(21708434)

In [44]:
(df 
 .select_dtypes(object)
 .head()
 )

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,Customer_City,Product_category,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift
0,7e9ae164-6a8b-4521-a2d4-58f7c9fff13f,Outcall,Product Queries,Life Insurance,NaN,c27c9bb4-fa36-4140-9f1f-21009254ffdb,NaN,01/08/2023 11:13,01/08/2023 11:47,01-Aug-23,NaN,NaN,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning
1,b07ec1b0-f376-43b6-86df-ec03da3b2e16,Outcall,Product Queries,Product Specific Information,NaN,d406b0c7-ce17-4654-b9de-f08d421254bd,NaN,01/08/2023 12:52,01/08/2023 12:54,01-Aug-23,NaN,NaN,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning
2,200814dd-27c7-4149-ba2b-bd3af3092880,Inbound,Order Related,Installation/demo,NaN,c273368d-b961-44cb-beaf-62d6fd6c00d5,NaN,01/08/2023 20:16,01/08/2023 20:38,01-Aug-23,NaN,NaN,Duane Norman,Jackson Park,William Kim,On Job Training,Evening
3,eb0d3e53-c1ca-42d3-8486-e42c8d622135,Inbound,Returns,Reverse Pickup Enquiry,NaN,5aed0059-55a4-4ec6-bb54-97942092020a,NaN,01/08/2023 20:56,01/08/2023 21:16,01-Aug-23,NaN,NaN,Patrick Flores,Olivia Wang,John Smith,>90,Evening
4,ba903143-1e54-406c-b969-46c52f92e5df,Inbound,Cancellation,Not Needed,NaN,e8bed5a9-6933-4aff-9dc6-ccefd7dcde59,NaN,01/08/2023 10:30,01/08/2023 10:32,01-Aug-23,NaN,NaN,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning


Customer remarks, issue_reported at and issue_responded_at , agent_name is left...

In [52]:

# agent name
# Since we are using this dataset for model training, we can simply drop unique id , order_id and order_date_time. Also droping customer 
    # remarks here, since we can't use it for our predictive modeling
(df
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time','Customer Remarks'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
 )

np.int64(12157720)

In [53]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 20 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Unique id                85907 non-null  object 
 1   channel_name             85907 non-null  object 
 2   category                 85907 non-null  object 
 3   Sub-category             85907 non-null  object 
 4   Customer Remarks         28742 non-null  object 
 5   Order_id                 67675 non-null  object 
 6   order_date_time          17214 non-null  object 
 7   Issue_reported at        85907 non-null  object 
 8   issue_responded          85907 non-null  object 
 9   Survey_response_Date     85907 non-null  object 
 10  Customer_City            17079 non-null  object 
 11  Product_category         17196 non-null  object 
 12  Item_price               17206 non-null  float64
 13  connected_handling_time  242 non-null    float64
 14  Agent_name            

In [57]:
# issue reported at and issue responded to

(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format = '%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format = '%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: ((d['issue_responded']-d['Issue_reported_at']).dt.total_seconds() / 3600).round(2)
   )
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time','Customer Remarks', 'Issue_reported at', 'issue_responded', 'Issue_reported_at'])
#  .memory_usage(deep=True)
#  .sum()    #  was 96483983
 .info()
 )

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   channel_name   85907 non-null  category
 1   category       85907 non-null  category
 2   Sub-category   85907 non-null  category
 3   Agent_name     85907 non-null  category
 4   Supervisor     85907 non-null  category
 5   Manager        85907 non-null  category
 6   Tenure Bucket  85907 non-null  category
 7   Agent Shift    85907 non-null  category
 8   CSAT Score     85907 non-null  int8    
 9   Respond_time   85907 non-null  float64 
dtypes: category(8), float64(1), int8(1)
memory usage: 1.5 MB


In [58]:
# we have still to decide datatype for new feature respond time

(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format = '%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format = '%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: ((d['issue_responded']-d['Issue_reported_at']).dt.total_seconds() / 3600).round(2)
   )
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category'})
#  .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
#                 'Customer_City','Unique id','Order_id','order_date_time','Customer Remarks','Issue_reported at','issue_responded','Issue_reported_at'])
#  .memory_usage(deep=True)
#  .sum()    #  was 96483983
#   .info()
# .select_dtypes(float)
.query("Respond_time < 0")
# .describe()
 )

,Unique id,channel_name,category,Sub-category,Customer Remarks,Order_id,order_date_time,Issue_reported at,issue_responded,Survey_response_Date,...,Item_price,connected_handling_time,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score,Issue_reported_at,Respond_time
74172,8a3b660a-3a3d-4761-8ccc-553c7e82c592,Inbound,Refund Related,COD Refund Details,NaN,01f44c10-63f5-49a5-9bba-54515be67974,24/07/2023 07:33,28/08/2023 16:07,2023-08-28,28-Aug-23,...,323.0,NaN,Steven Wall,Zoe Yamamoto,Emily Chen,>90,Afternoon,5,2023-08-28 16:07:00,-16.12
74173,4f1e4d87-44f8-4a6e-9952-385e27ccc4ae,Outcall,Returns,Reverse Pickup Enquiry,NaN,d677c3d9-20b3-4fea-802c-22e980851f60,NaN,28/08/2023 16:56,2023-08-28,28-Aug-23,...,NaN,NaN,Brittney Key,Mia Yamamoto,Jennifer Nguyen,On Job Training,Morning,3,2023-08-28 16:56:00,-16.93
74174,b8f28222-bf9d-4509-9b2b-6e36d10c9e87,Inbound,Payments related,Online Payment Issues,NaN,46ff1a3e-066c-460b-97f3-640f665fa171,NaN,28/08/2023 20:14,2023-08-28,28-Aug-23,...,NaN,NaN,Brian Koch,Ethan Tan,Jennifer Nguyen,On Job Training,Evening,5,2023-08-28 20:14:00,-20.23
74175,6405d511-6679-4cf6-a243-ff4ea7e303d9,Inbound,Payments related,Billing Related,NaN,cb1c512d-abe3-4f98-893c-f11cab8bd0f1,NaN,28/08/2023 14:37,2023-08-28,28-Aug-23,...,NaN,NaN,Ronnie Romero,Aiden Patel,John Smith,>90,Morning,5,2023-08-28 14:37:00,-14.62
74176,5b424642-5c4c-42d9-8dc9-750413cc2777,Inbound,Returns,Return request,NaN,967b5c93-1e52-42e5-b9ff-15b4e6cf8d8e,NaN,28/08/2023 23:26,2023-08-28,28-Aug-23,...,NaN,NaN,Ann English,Olivia Suzuki,Olivia Tan,On Job Training,Morning,5,2023-08-28 23:26:00,-23.43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77566,7bac7fcd-8ae1-46c0-9315-dc42325e5102,Outcall,Payments related,Payment pending,So glad,979360d0-c159-4b38-8362-3988cdc5f100,NaN,28/08/2023 13:25,2023-08-28,28-Aug-23,...,NaN,NaN,Tammy Gould,Ava Wong,Jennifer Nguyen,On Job Training,Morning,5,2023-08-28 13:25:00,-13.42
77567,95d5fa75-b04a-4da1-96c9-1bdfcedf107e,Outcall,Returns,Return request,NaN,2887bf70-b149-49d2-a947-353e59db4ffe,NaN,28/08/2023 18:09,2023-08-28,28-Aug-23,...,NaN,NaN,Cathy Harris,Charlotte Suzuki,William Kim,On Job Training,Morning,5,2023-08-28 18:09:00,-18.15
77568,771dace8-342b-4551-ba27-ea21befe67fc,Outcall,Returns,Reverse Pickup Enquiry,Good,323e1981-3b3d-4f79-8108-7ddac1f8267b,NaN,28/08/2023 22:20,2023-08-28,28-Aug-23,...,NaN,NaN,Jessica Cook,Ethan Tan,Michael Lee,>90,Evening,4,2023-08-28 22:20:00,-22.33
77569,ccc21c3e-97a9-41e9-89f2-65a5c7e609d8,Inbound,Order Related,Order status enquiry,Your delivery agents are not doing good work...,1121dc82-f80b-48f4-8282-9359d7a195c3,NaN,28/08/2023 14:38,2023-08-28,28-Aug-23,...,NaN,NaN,Madison Flores,Ethan Tan,Jennifer Nguyen,On Job Training,Evening,1,2023-08-28 14:38:00,-14.63


We need to perform 2 operation, first understand reason for negative respond_time and handle it &&  convert float64 to float8.(since max value is <255 )

In [59]:
# get the df with negative repond time
(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format = '%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format = '%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: ((d['issue_responded']-d['Issue_reported_at']).dt.total_seconds() / 3600).round(2).clip(lower=0)
   )
# .assign(
#     Respond_time = lambda d: np.where(d['Respond_time']<0,0,d['Respond_time'])
# )
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time','Issue_reported at','issue_responded','Issue_reported_at'])
#  .memory_usage(deep=True)
#  .sum()    #  was 96483983
#   .info()
.select_dtypes(float)
.describe()
# .query("Respond_time < 0")

 )

,Respond_time
count,85907.000000
mean,2.827683
std,9.263143
min,0.000000
25%,0.030000
50%,0.080000
75%,0.580000
max,95.970000


In [61]:
# get the repond time data type
(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format = '%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format = '%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: ((d['issue_responded']-d['Issue_reported_at']).dt.total_seconds() / 3600).round(2).clip(lower=0)
   )
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category','Respond_time':'float32'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time','Issue_reported at','issue_responded','Issue_reported_at'])
 .memory_usage(deep=True)
 .sum()    #  was 96483983
)


np.int64(5824924)

In [62]:
import pandas as pd

def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    return (
        df
        .assign(
            Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format='%d/%m/%Y %H:%M'),
            issue_responded = pd.to_datetime(df['issue_responded'], format='%d/%m/%Y %H:%M')
        )
        .assign(
            Respond_time = lambda d: ((d['issue_responded'] - d['Issue_reported_at']).dt.total_seconds() / 3600)
                                     .round(2)
                                     .clip(lower=0)
        )
        .astype({
            'CSAT Score': 'int8',
            'channel_name': 'category',
            'category': 'category',
            'Sub-category': 'category',
            'Supervisor': 'category',
            'Manager': 'category',
            'Agent Shift': 'category',
            'Tenure Bucket': 'category',
            'Agent_name': 'category',
            'Respond_time': 'float32'
        })
        .drop(columns=[
            'Item_price', 'connected_handling_time', 'Survey_response_Date', 'Product_category',
            'Customer_City', 'Unique id', 'Order_id', 'order_date_time','Customer Remarks',
            'Issue_reported at', 'issue_responded', 'Issue_reported_at'
        ])
    )

In [64]:

df = pd.read_csv("Customer_support_data.csv")
processed_df = preprocess_data(df)
memory_used = processed_df.memory_usage(deep=True).sum()
print(f"Memory used: {memory_used}")

Memory used: 1333438


In [65]:
processed_df.head()

,channel_name,category,Sub-category,Agent_name,Supervisor,Manager,Tenure Bucket,Agent Shift,CSAT Score,Respond_time
0,Outcall,Product Queries,Life Insurance,Richard Buchanan,Mason Gupta,Jennifer Nguyen,On Job Training,Morning,5,0.57
1,Outcall,Product Queries,Product Specific Information,Vicki Collins,Dylan Kim,Michael Lee,>90,Morning,5,0.03
2,Inbound,Order Related,Installation/demo,Duane Norman,Jackson Park,William Kim,On Job Training,Evening,5,0.37
3,Inbound,Returns,Reverse Pickup Enquiry,Patrick Flores,Olivia Wang,John Smith,>90,Evening,5,0.33
4,Inbound,Cancellation,Not Needed,Christopher Sanchez,Austin Johnson,Michael Lee,0-30,Morning,5,0.03


In [66]:
processed_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   channel_name   85907 non-null  category
 1   category       85907 non-null  category
 2   Sub-category   85907 non-null  category
 3   Agent_name     85907 non-null  category
 4   Supervisor     85907 non-null  category
 5   Manager        85907 non-null  category
 6   Tenure Bucket  85907 non-null  category
 7   Agent Shift    85907 non-null  category
 8   CSAT Score     85907 non-null  int8    
 9   Respond_time   85907 non-null  float32 
dtypes: category(8), float32(1), int8(1)
memory usage: 1.2 MB


In [77]:

processed_df.to_parquet("cleaned_customer_support_data.parquet", index=False)

In [2]:
df_new = pd.read_parquet("cleaned_customer_support_data.parquet")
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85907 entries, 0 to 85906
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   channel_name   85907 non-null  category
 1   category       85907 non-null  category
 2   Sub-category   85907 non-null  category
 3   Agent_name     85907 non-null  category
 4   Supervisor     85907 non-null  category
 5   Manager        85907 non-null  category
 6   Tenure Bucket  85907 non-null  category
 7   Agent Shift    85907 non-null  category
 8   CSAT Score     85907 non-null  int8    
 9   Respond_time   85907 non-null  float32 
dtypes: category(8), float32(1), int8(1)
memory usage: 1.2 MB


# Feature Engg 

**After achieving max 0.555 auc roc; we are trying to find better features to represent our model.**

In [ ]:
df = pd.read_csv('Customer_support_data.csv')
df

In [ ]:
# get the repond time data type
(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format = '%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format = '%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: ((d['issue_responded']-d['Issue_reported_at']).dt.total_seconds() / 3600).round(2).clip(lower=0)
   )
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category','Respond_time':'float32'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time','Issue_reported at','issue_responded','Issue_reported_at'])
#  .memory_usage(deep=True)
#  .sum()    #  was 96483983
# .describe()
)

In [ ]:
# get the repond time data type
(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format = '%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format = '%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: ((d['issue_responded']-d['Issue_reported_at']).dt.total_seconds() / 3600).round(2).clip(lower=0)
   )
 .astype({'CSAT Score':'int8', 'channel_name':'category','category':'category','Sub-category':'category'  , 'Supervisor':'category',
          'Manager':'category', 'Agent Shift': 'category', 'Tenure Bucket': 'category', 'Agent_name':'category','Respond_time':'float32'})
 .drop(columns=['Item_price','connected_handling_time','Survey_response_Date','Product_category',
                'Customer_City','Unique id','Order_id','order_date_time','Issue_reported at','issue_responded','Issue_reported_at'])
#  .memory_usage(deep=True)
#  .sum()    #  was 96483983
# .describe()
)['Respond_time'].quantile([0.75, 0.8 , 0.9, 1])

Let make 3 more categorical features from respond_time; one as respond time every 6 hrs...keep 17 labels of this .. 16 for first 4 days ... one as remaining days.... other as 5 labels ... 4 for first 4 days ... 5 th for rest....  and last as true for respond time < 1 and false for rest....

In [ ]:
(df
 .assign(
     Issue_reported_at = pd.to_datetime(df['Issue_reported at'], format='%d/%m/%Y %H:%M'),
     issue_responded = pd.to_datetime(df['issue_responded'], format='%d/%m/%Y %H:%M')
 )
 .assign(
     Respond_time = lambda d: (
         (d['issue_responded'] - d['Issue_reported_at']).dt.total_seconds() / 3600
     ).round(2).clip(lower=0)
 )
 .assign(
     Respond_time_6hr_bucket = lambda d: d['Respond_time'].apply(
         lambda x: int(min(x // 6, 16))  # 0–15 buckets for 0–96h, 16th bucket for >96h
     ).astype('int8'),

     Respond_time_day_bucket = lambda d: d['Respond_time'].apply(
         lambda x: int(min(x // 24, 4))  # 0–3 for 0–96h, 4th for >96h
     ).astype('int8'),

     Respond_time_lt_1hr = lambda d: (d['Respond_time'] < 1)
 )
 .astype({
     'CSAT Score': 'int8',
     'channel_name': 'category',
     'category': 'category',
     'Sub-category': 'category',
     'Supervisor': 'category',
     'Manager': 'category',
     'Agent Shift': 'category',
     'Tenure Bucket': 'category',
     'Agent_name': 'category',
     'Respond_time': 'float32',
     'Respond_time_6hr_bucket': 'category',
     'Respond_time_day_bucket': 'category',
     'Respond_time_lt_1hr': 'bool'
 })
 .drop(columns=[
     'Item_price', 'connected_handling_time', 'Survey_response_Date', 'Product_category',
     'Customer_City', 'Unique id', 'Order_id', 'order_date_time',
     'Issue_reported at', 'issue_responded', 'Issue_reported_at'
 ])
)


***Engineered More Features***

In [ ]:
import pandas as pd
import numpy as np

def preprocess_customer_support_data(df: pd.DataFrame) -> pd.DataFrame:
    df = (
        df
        .assign(
            Issue_reported_at=pd.to_datetime(df['Issue_reported at'], format='%d/%m/%Y %H:%M'),
            issue_responded=pd.to_datetime(df['issue_responded'], format='%d/%m/%Y %H:%M')
        )
        .assign(
            Respond_time=lambda d: (
                (d['issue_responded'] - d['Issue_reported_at']).dt.total_seconds() / 3600
            ).round(2).clip(lower=0),
            Issue_hour=lambda d: d['Issue_reported_at'].dt.hour,
            Issue_dayofweek=lambda d: d['Issue_reported_at'].dt.dayofweek,
            Agent_ticket_volume=lambda d: d.groupby('Agent_name')['Agent_name'].transform('count'),
            channel_shift=lambda d: d['channel_name'].astype(str) + '_' + d['Agent Shift'].astype(str),
            log_Respond_time=lambda d: np.log1p(d['Respond_time']),
            Respond_time_lt_1hr=lambda d: (d['Respond_time'] < 1)
        )
        .assign(
            Item_price_category=lambda d: np.select(
                [
                    d['Item_price'].isna(),
                    d['Item_price'] < 500,
                    (d['Item_price'] >= 500) & (d['Item_price'] < 5000),
                    d['Item_price'] >= 5000
                ],
                [
                    'missing', 'low', 'medium', 'high'
                ],
                default='missing'
            )
        )
        .assign(
            order_date_time=pd.to_datetime(df['order_date_time'], format='%d/%m/%Y %H:%M', errors='coerce'),
            Query_raise_time_days=lambda d: np.where(
                d['order_date_time'].notna(),
                (d['Issue_reported_at'] - d['order_date_time']).dt.days.clip(lower=0),
                100
            )
        )
        .astype({
            'CSAT Score': 'int8',
            'channel_name': 'category',
            'category': 'category',
            'Sub-category': 'category',
            'Supervisor': 'category',
            'Manager': 'category',
            'Agent Shift': 'category',
            'Tenure Bucket': 'category',
            'Agent_name': 'category',
            'Respond_time': 'float32',
            'log_Respond_time': 'float32',
            'Respond_time_lt_1hr': 'bool',
            'Item_price_category': 'category',
            'channel_shift': 'category',
            'Issue_hour': 'int8',
            'Issue_dayofweek': 'int8',
            'Agent_ticket_volume': 'int16',
            'Query_raise_time_days': 'int32'
        })
        .drop(columns=[
            'Item_price', 'connected_handling_time', 'Survey_response_Date', 'Product_category',
            'Customer_City', 'Unique id', 'Order_id', 'order_date_time',
            'Issue_reported at', 'issue_responded', 'Issue_reported_at', 'Customer Remarks'
        ])
    )
    return df


In [ ]:
df = pd.read_csv('Customer_support_data.csv')
df = preprocess_customer_support_data(df)
df.to_parquet('featureEngg.parquet', index=False)

In [ ]:
df = pd.read_parquet('featureEngg.parquet')
df

# Data Visualizations

make very good plots, with insights with some business relations/context and take help from the kaggle notebook
and daily dose of DS 2024.